In [48]:
import pandas as pd
import numpy as np


In [49]:
# 0 = False, 1= True
data = pd.DataFrame({"X1" : [0,0,1,1],
             "X2" : [0,1,0,1],
             "Y" : [0,1,1,1]})

In [50]:
data

,X1,X2,Y
0,0,0,0
1,0,1,1
2,1,0,1
3,1,1,1


In [51]:
# initial_entropy rootnode
def entropy_calculation(y):
    entropy = 0
    for i in set(y):
#         print(f"label class count {i}:",sum(y == i))
        entropy -= ((sum(y == i) / len(y)) * np.log2(sum(y == i) / len(y)))  
    return entropy

In [52]:
def weighted_entropy_calculation(df, y):
    df = data["X1"] # taking one feature
    weighted_entropy_for_feature = 0
    entropy_of_feature = 0
    for labels in set(df): # check the labels in the X1 feature
        # need to filter the target records w.r.t X1
        new_y_with_repect_to_labels = (y[df == labels])
        entropy_of_each_label_in_feature = entropy_calculation(new_y_with_repect_to_labels)
        entropy_of_feature += entropy_of_each_label_in_feature
        # calculate the weighted entropy overall
        weighted_entropy_for_feature += (new_y_with_repect_to_labels).count() / len(y) * entropy_of_each_label_in_feature
        
    return weighted_entropy_for_feature, entropy_of_feature
    
    
    
    

In [53]:
def each_level_information(initial_gain_before_taking_that_feature, df, y):
    weighted_entropy_for_feature, entropy_of_feature = weighted_entropy_calculation(df, y)
    # gain = parent_entropy(entropy before taking the feature) - entropy_at_each_feature
    gain = initial_gain_before_taking_that_feature - weighted_entropy_for_feature
#     print(gain)
    return gain, weighted_entropy_for_feature


In [54]:
def dt_implementation(df, y, unused_features, parent_entropy):
    #base case
    if (len(unused_features)==0) or (len(set(y))==1):
        return
    
    # need to initiate the best feature for the split
    best_feature = ""
    final_gain = -1
    final_entropy = 0
    
    # need to calculate the entropy and gain ratio for all the feature best one need to choose for splitting criteria
    for feature in unused_features:
        gain, weighted_entropy_for_feature = each_level_information(parent_entropy, df[feature], y)
        if gain > final_gain:
            final_gain=gain
            best_feature=feature
            final_entropy=weighted_entropy_for_feature
            
    unused_features.remove(best_feature)
    # print it out
    for i in set(y):
        print("Count of", i, "=", sum(y==i))
    print("Current Entropy is =", final_entropy)
    if final_entropy==0:
        print('Reached leaf node')
    else:
        print("Splitting on feature", best_feature, "with gain ratio", final_gain)
    print()
    
    # recurvelly call after splitting with that feature
    for labels in set(df[best_feature]):
        dt_implementation(df[df[best_feature] == labels], y[df[best_feature] == labels], unused_features, final_entropy)

In [55]:
y = (data.Y)
unused_features = set(data.columns)
df = data.drop('Y', axis=1)
parent_entropy = entropy_calculation(y)

In [56]:
dt_implementation(data, y, unused_features, parent_entropy)

Count of 0 = 1
Count of 1 = 3
Current Entropy is = 0.5
Splitting on feature X2 with gain ratio 0.31127812445913283

Count of 0 = 1
Count of 1 = 1
Current Entropy is = 0.0
Reached leaf node

